In [1]:
import json
import os
import re
import time
import uuid
import pandas as pd
import requests
from Crypto.Hash import MD5
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill, Font
from playwright.sync_api import sync_playwright
from loguru import logger
import datetime
import arrow
import shutil
import openpyxl as op
import numpy as np

In [2]:
data_dict = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M',
             14: 'N', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y',
             26: 'Z', 0: ''}
listn = []
for t in range(1,100):
    p = t//26
    q = t%26
    col_name = data_dict[p] + data_dict[q]
    if q == 0 and p > 0:
        col_name = data_dict[p - 1] + 'Z'
    listn.append(col_name)
print(listn)
data_file = r'C:\Users\EDY\Desktop\郑明导入模板\总部及各事业部项目字段对应表(2).xls'  # 字段
sheet_name1 = '薪资规则2（冷链）'
data_file2 = r'C:\Users\EDY\Desktop\郑明导入模板\总部及各事业部工资线下表模板(1).xlsx'  # 线下表
sheet_name2 = '冷链物流事业部'
df = pd.read_excel(data_file, header=1, sheet_name=sheet_name1)  # 字段
df1 = pd.read_excel(data_file, header=4, sheet_name=sheet_name1)
print('df1', df1.columns)
df_1 = pd.read_excel(data_file2, header=1, sheet_name=sheet_name2)# 线下表的列名
df_2 = pd.read_excel(data_file2, header=2, sheet_name=sheet_name2)
print('df_1', df_1.columns)
list_no = [x for x in df1.columns if x not in df_1.columns and x not in df_2.columns and 'Unnamed' not in x]
list1 = []
list2 = []
list_compare = []
t = 0
for i, j in zip(df.columns, df1.columns):
    t = t+1
    if '备注' == i or t == 1:
        continue
    elif t == 2:
        a = ['姓名', t, 1]
        list1.append(a)
        print(a, '*********', 'B')
        continue
    elif t == 3:
        a = ['身份证号', t, 1]
        list1.append(a)
        print(a, '*********', 'C')
        continue
    if 'Unnamed' in j:
        a = [i, t]
        list2.append(a)
        print(a)
        continue
    else:
        if j in df_1.columns and j in df_2.columns:
            ln = '?'
        elif j in df_1.columns:
            ln = 1
        elif j in df_2.columns:
            ln = 2
        else:
            ln = '?'
        a = [j, t, ln]
        list1.append(a)
        p = t//26
        q = t%26
        col_name = data_dict[p] + data_dict[q]
        if q == 0 and p > 0:
            col_name = data_dict[p - 1] + 'Z'
        print(a, '*********', col_name, '*****', ln)
        list_compare.append(col_name)
        continue
print('\n', sheet_name2, '\n')
print('list1 =', list1)
print('list2 =', list2)
print('list_compare =', list_compare)
print('没有在线下表的list_no=', list_no)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU']
df1 Index(['Unnamed: 0', 'Unnamed: 1', '校验人员重复性', 'Unnamed: 3', 'Unnamed: 4',
       '基本工资', '岗位工资', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       '应出勤', '出勤天数', '迟到', '调休、年假、公休', '事假', '病假', '缺勤', 'Unnamed: 18',
       'Unnamed: 19', '货损扣款', '事假/缺勤', '病假.1', 'Unnamed: 23', '其他扣款', '补贴',
       '加班', 'Unnamed: 27', 'Unnamed: 28', '高温津贴（餐费）', 'Unnamed: 30', '公里数补贴',
       '服务奖', '通讯费', '安全奖', 'Unnamed: 35', 'U

In [3]:
"""x，类array对象，且必须为一维，待切割的原形式
bins, 整数、序列尺度、或间隔索引。如果bins是一个整数，它定义了x宽度范围内的等宽面元数量，但是在这种情况下，x的范围在每个边上被延长1%，以保证包括x的最小值或最大值。如果bin是序列，它定义了允许非均匀bin宽度的bin边缘。在这种情况下没有x的范围的扩展。
right,布尔值。是否是左开右闭区间
labels,用作结果箱的标签。必须与结果箱相同长度。如果FALSE，只返回整数指标面元。
retbins,布尔值。是否返回面元
precision，整数。返回面元的小数点几位
include_lowest，布尔值。第一个区间的左端点是否包含"""
print('\n', pd.cut(np.array([0.2,1.4,2.5,6.2,9.7,2.1]), 3, retbins=True))
print('\n', pd.cut(np.array([0.2,1.4,2.5,6.2,9.7,2.1]), 3, labels=["good", "medium", "bad"]))
print('\n', pd.cut(np.ones(5), 3, labels=False))
ages = np.array([1,5,10,40,36,12,58,62,77,89,100,18,20,25,30,32]) #年龄数据
print('\n', pd.cut(ages, 5, labels=[u"婴儿",u"青年",u"中年",u"壮年",u"老年"]))
#若labels为False则返回整数填充的Categorical或数组或Series
#若retbins为True还返回用浮点数填充的N维数组
print('\n', pd.qcut(range(7), 3))
#结果中超过边界的值将会变成NA
a = np.array([0.2,1.4,2.5,6.2,9.7,2.1])
print(a)
print('\n', pd.cut([0.2,1.4,2.5,6.2,9.7,4.1], 3, retbins=True))


 ([(0.19, 3.367], (0.19, 3.367], (0.19, 3.367], (3.367, 6.533], (6.533, 9.7], (0.19, 3.367]]
Categories (3, interval[float64, right]): [(0.19, 3.367] < (3.367, 6.533] < (6.533, 9.7]], array([0.1905    , 3.36666667, 6.53333333, 9.7       ]))

 ['good', 'good', 'good', 'medium', 'bad', 'good']
Categories (3, object): ['good' < 'medium' < 'bad']

 [1 1 1 1 1]

 ['婴儿', '婴儿', '婴儿', '青年', '青年', ..., '婴儿', '婴儿', '青年', '青年', '青年']
Length: 16
Categories (5, object): ['婴儿' < '青年' < '中年' < '壮年' < '老年']

 [(-0.001, 2.0], (-0.001, 2.0], (-0.001, 2.0], (2.0, 4.0], (2.0, 4.0], (4.0, 6.0], (4.0, 6.0]]
Categories (3, interval[float64, right]): [(-0.001, 2.0] < (2.0, 4.0] < (4.0, 6.0]]
[0.2 1.4 2.5 6.2 9.7 2.1]

 ([(0.19, 3.367], (0.19, 3.367], (0.19, 3.367], (3.367, 6.533], (6.533, 9.7], (3.367, 6.533]]
Categories (3, interval[float64, right]): [(0.19, 3.367] < (3.367, 6.533] < (6.533, 9.7]], array([0.1905    , 3.36666667, 6.53333333, 9.7       ]))


In [4]:
d = pd.DataFrame({
    '城市':['沈阳','深圳','广州','上海','北京'],
    '同比':[101.4,140.9,101.4,127.3,120.7],
    '环比':[100.1,102.0,101.3,101.2,101.5],
    '定基':[101.6,145.6,120.0,127.8,121.4]
}, index=['c5','c4','c3','c2','c1'])
print(d)
print('d.columns:',d.columns)
print('d.index:',d.index)

    城市     同比     环比     定基
c5  沈阳  101.4  100.1  101.6
c4  深圳  140.9  102.0  145.6
c3  广州  101.4  101.3  120.0
c2  上海  127.3  101.2  127.8
c1  北京  120.7  101.5  121.4
d.columns: Index(['城市', '同比', '环比', '定基'], dtype='object')
d.index: Index(['c5', 'c4', 'c3', 'c2', 'c1'], dtype='object')


In [5]:
print(d['城市'])
d['预测'] = 16.5
print('\n预测\n', d)

c5    沈阳
c4    深圳
c3    广州
c2    上海
c1    北京
Name: 城市, dtype: object

预测
     城市     同比     环比     定基    预测
c5  沈阳  101.4  100.1  101.6  16.5
c4  深圳  140.9  102.0  145.6  16.5
c3  广州  101.4  101.3  120.0  16.5
c2  上海  127.3  101.2  127.8  16.5
c1  北京  120.7  101.5  121.4  16.5


In [6]:
val = pd.Series([-1.2,-1.5,-1.7], index=['c5','c3','c2'])
d['预测'] = val
print('预测val\n', d)

预测val
     城市     同比     环比     定基   预测
c5  沈阳  101.4  100.1  101.6 -1.2
c4  深圳  140.9  102.0  145.6  NaN
c3  广州  101.4  101.3  120.0 -1.5
c2  上海  127.3  101.2  127.8 -1.7
c1  北京  120.7  101.5  121.4  NaN


In [7]:
d['判断环比'] = d['环比'] > 101.2
print('\n判断环比\n', d)
del d['预测']
print("\ndel d['预测']\n",d)


判断环比
     城市     同比     环比     定基   预测   判断环比
c5  沈阳  101.4  100.1  101.6 -1.2  False
c4  深圳  140.9  102.0  145.6  NaN   True
c3  广州  101.4  101.3  120.0 -1.5   True
c2  上海  127.3  101.2  127.8 -1.7  False
c1  北京  120.7  101.5  121.4  NaN   True

del d['预测']
     城市     同比     环比     定基   判断环比
c5  沈阳  101.4  100.1  101.6  False
c4  深圳  140.9  102.0  145.6   True
c3  广州  101.4  101.3  120.0   True
c2  上海  127.3  101.2  127.8  False
c1  北京  120.7  101.5  121.4   True


In [8]:
ni = d.index.insert(5,'c0')
nc = d.columns.insert(3,'»å(无法识别)')
print('\n»å(无法识别)\n', nc)
nd = d.reindex(index=ni, columns=nc).ffill()
print('\n', nd)
print('\n', nd.sort_index())


»å(无法识别)
 Index(['城市', '同比', '环比', '»å(无法识别)', '定基', '判断环比'], dtype='object')

     城市     同比     环比  »å(无法识别)     定基   判断环比
c5  沈阳  101.4  100.1       NaN  101.6  False
c4  深圳  140.9  102.0       NaN  145.6   True
c3  广州  101.4  101.3       NaN  120.0   True
c2  上海  127.3  101.2       NaN  127.8  False
c1  北京  120.7  101.5       NaN  121.4   True
c0  北京  120.7  101.5       NaN  121.4   True

     城市     同比     环比  »å(无法识别)     定基   判断环比
c0  北京  120.7  101.5       NaN  121.4   True
c1  北京  120.7  101.5       NaN  121.4   True
c2  上海  127.3  101.2       NaN  127.8  False
c3  广州  101.4  101.3       NaN  120.0   True
c4  深圳  140.9  102.0       NaN  145.6   True
c5  沈阳  101.4  100.1       NaN  101.6  False


In [9]:
nd.iloc[4,2] = 102.0
nd.iloc[4,3] = 102.0
nd.loc['c4','»å(无法识别)'] = -1.2
print('\n', nd.sort_values('同比'))
nd.drop(['»å(无法识别)'],axis=1,inplace=True)


     城市     同比     环比  »å(无法识别)     定基   判断环比
c5  沈阳  101.4  100.1       NaN  101.6  False
c3  广州  101.4  101.3       NaN  120.0   True
c1  北京  120.7  102.0     102.0  121.4   True
c0  北京  120.7  101.5       NaN  121.4   True
c2  上海  127.3  101.2       NaN  127.8  False
c4  深圳  140.9  102.0      -1.2  145.6   True


In [10]:
nf = nd.groupby('判断环比')
print('\n', nf.groups)
print(len(nf.groups))
for i, j in nf:
    print(i)
    print(j,'\n')
print('\nFalse\n', nf.get_group(False))
print('\nTrue\n', nf.get_group(True))
print('\nsum\n', nf.sum())
print('\nsum\n', nf[['同比', '环比']].agg('sum'))
print("\n'sum', 'mean', 'std'\n", nf[['同比', '环比']].agg(['sum', 'mean', 'std']))
print('\n', nf[['同比', '环比']].agg([np.sum, np.mean, np.std]))
print('\n', nf.describe())


 {False: ['c5', 'c2'], True: ['c4', 'c3', 'c1', 'c0']}
2
False
    城市     同比     环比     定基   判断环比
c5  沈阳  101.4  100.1  101.6  False
c2  上海  127.3  101.2  127.8  False 

True
    城市     同比     环比     定基  判断环比
c4  深圳  140.9  102.0  145.6  True
c3  广州  101.4  101.3  120.0  True
c1  北京  120.7  102.0  121.4  True
c0  北京  120.7  101.5  121.4  True 


False
     城市     同比     环比     定基   判断环比
c5  沈阳  101.4  100.1  101.6  False
c2  上海  127.3  101.2  127.8  False

True
     城市     同比     环比     定基  判断环比
c4  深圳  140.9  102.0  145.6  True
c3  广州  101.4  101.3  120.0  True
c1  北京  120.7  102.0  121.4  True
c0  北京  120.7  101.5  121.4  True

sum
           同比     环比     定基
判断环比                      
False  228.7  201.3  229.4
True   483.7  406.8  508.4

sum
           同比     环比
判断环比               
False  228.7  201.3
True   483.7  406.8

'sum', 'mean', 'std'
           同比                         环比                  
         sum     mean        std    sum    mean       std
判断环比                   

In [11]:
nf = nd.groupby(['判断环比','同比'])
print("\n'判断环比','同比'\n", nf.sum())
print(len(nf.groups))
for i, j in nf:
    print(i)
    print(j)
print('\nnf.size()\n', nf.size())
print('\nnf.mean()\n', nf.mean())
print('\nnf.median()\n', nf.median())


'判断环比','同比'
                 环比     定基
判断环比  同比                 
False 101.4  100.1  101.6
      127.3  101.2  127.8
True  101.4  101.3  120.0
      120.7  203.5  242.8
      140.9  102.0  145.6
5
(False, 101.4)
    城市     同比     环比     定基   判断环比
c5  沈阳  101.4  100.1  101.6  False
(False, 127.3)
    城市     同比     环比     定基   判断环比
c2  上海  127.3  101.2  127.8  False
(True, 101.4)
    城市     同比     环比     定基  判断环比
c3  广州  101.4  101.3  120.0  True
(True, 120.7)
    城市     同比     环比     定基  判断环比
c1  北京  120.7  102.0  121.4  True
c0  北京  120.7  101.5  121.4  True
(True, 140.9)
    城市     同比     环比     定基  判断环比
c4  深圳  140.9  102.0  145.6  True

nf.size()
 判断环比   同比   
False  101.4    1
       127.3    1
True   101.4    1
       120.7    2
       140.9    1
dtype: int64

nf.mean()
                  环比     定基
判断环比  同比                  
False 101.4  100.10  101.6
      127.3  101.20  127.8
True  101.4  101.30  120.0
      120.7  101.75  121.4
      140.9  102.00  145.6

nf.median()
          

In [12]:
valtime = pd.Series(['1999-01-22', '1999-01-22', '1999-01-27', '1999-01-27', '1999-01-28', '1999-01-28'],
                    index=['c5','c4','c3','c2','c1','c0'])
nd['Date'] = pd.to_datetime(valtime)
print('\nDate\n', nd)
nd['year'] = nd['Date'].map(lambda x:x.year)
nd.index = nd['Date']
nd.drop(['Date'],axis=1,inplace=True)
del nd['城市']
print("\nnd.index = nd['Date']\n", nd)
print('\nnd.head\n', nd.head(3))
print('\nnd.tail\n', nd.tail(3))


Date
     城市     同比     环比     定基   判断环比       Date
c5  沈阳  101.4  100.1  101.6  False 1999-01-22
c4  深圳  140.9  102.0  145.6   True 1999-01-22
c3  广州  101.4  101.3  120.0   True 1999-01-27
c2  上海  127.3  101.2  127.8  False 1999-01-27
c1  北京  120.7  102.0  121.4   True 1999-01-28
c0  北京  120.7  101.5  121.4   True 1999-01-28

nd.index = nd['Date']
                同比     环比     定基   判断环比  year
Date                                        
1999-01-22  101.4  100.1  101.6  False  1999
1999-01-22  140.9  102.0  145.6   True  1999
1999-01-27  101.4  101.3  120.0   True  1999
1999-01-27  127.3  101.2  127.8  False  1999
1999-01-28  120.7  102.0  121.4   True  1999
1999-01-28  120.7  101.5  121.4   True  1999

nd.head
                同比     环比     定基   判断环比  year
Date                                        
1999-01-22  101.4  100.1  101.6  False  1999
1999-01-22  140.9  102.0  145.6   True  1999
1999-01-27  101.4  101.3  120.0   True  1999

nd.tail
                同比     环比     定基   判断环比  ye

In [13]:
def year_trans(x):
    return x-1
nd['day'] = nd.index.map(lambda x:x.day)
nf = nd.groupby('day').transform(year_trans)
print(nf)

               同比     环比     定基  判断环比  year
Date                                       
1999-01-22  100.4   99.1  100.6    -1  1998
1999-01-22  139.9  101.0  144.6     0  1998
1999-01-27  100.4  100.3  119.0     0  1998
1999-01-27  126.3  100.2  126.8    -1  1998
1999-01-28  119.7  101.0  120.4     0  1998
1999-01-28  119.7  100.5  120.4     0  1998
